In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import keras

In [ ]:
news_final=pd.read_csv("DataSet/News_Final.csv")

news_final.describe()
print(news_final.shape)

In [ ]:
news_final[["Facebook","GooglePlus","LinkedIn"]] = news_final[["Facebook","GooglePlus","LinkedIn"]] .replace(-1, np.NaN)
news_final.dropna(inplace=True)
print(news_final.shape)

In [ ]:
#Data cleaning
#removing unnecessary punctuations and tags

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    
    return input_txt

In [ ]:
news_final['Title'] = np.vectorize(remove_pattern)(news_final['Title'], "@[\\w]*")

#replacing numbers and other special characters with a space in the Title elements
news_final['Title'] = news_final['Title'].str.replace("[^a-zA-Z#]", " ")

news_final['Headline'] = np.vectorize(remove_pattern)(news_final['Headline'], "@[\\w]*")
news_final['Headline'] = news_final['Headline'].str.replace("[^a-zA-Z#]", " ")

#remove stopwords
stop = stopwords.words('english')
news_final= pd.DataFrame(news_final)
news_final['Title'] = news_final['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


stop = stopwords.words('english')
news_final= pd.DataFrame(news_final)
news_final['Headline'] = news_final['Headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [ ]:
news_final.dtypes

In [ ]:
z=[]
z=list(zip(news_final['IDLink'],news_final['Title'],news_final['Headline'],news_final['Source'],news_final['Topic'],news_final['Facebook']))
z1=sorted(z, key=lambda t: t[5])

labels = ['IDLink','Title', 'Headline','Source', 'Topic', 'SharesOfFacebook']
df_facebook = pd.DataFrame.from_records(z1, columns=labels)

def label_emotion (row):
   if row['SharesOfFacebook'] >=0 and row['SharesOfFacebook'] <=1050  :
      return 1
   if row['SharesOfFacebook'] > 1050 and row['SharesOfFacebook']<=15000:
      return 2
   if row['SharesOfFacebook'] > 15000 and row['SharesOfFacebook']<=20000:
      return 3
   if row['SharesOfFacebook'] >20000 and row['SharesOfFacebook'] <=50000:
      return 4
df_facebook['category_facebook'] = df_facebook.apply (lambda row: label_emotion (row),axis=1)

df_facebook

In [ ]:
z=[]
z=list(zip(news_final['IDLink'],news_final['Title'],news_final['Headline'],news_final['Source'],news_final['Topic'],news_final['GooglePlus']))

z1=sorted(z, key=lambda t: t[5])

labels = ['IDLink','Title', 'Headline','Source', 'Topic', 'SharesOfGooglePlus']
df_googlePlus = pd.DataFrame.from_records(z1, columns=labels)

def label_emotion (row):
   if row['SharesOfGooglePlus'] >=0 and row['SharesOfGooglePlus'] <=40  :
      return 1
   if row['SharesOfGooglePlus'] > 40 and row['SharesOfGooglePlus']<=120:
      return 2
   if row['SharesOfGooglePlus'] > 120 and row['SharesOfGooglePlus']<=600:
      return 3
   if row['SharesOfGooglePlus'] >600 and row['SharesOfGooglePlus'] <=605:
      return 4
   if row['SharesOfGooglePlus'] >605 and row['SharesOfGooglePlus'] <=1500:
      return 5

df_googlePlus['category_GooglePlus'] = df_googlePlus.apply (lambda row: label_emotion (row),axis=1)

df_googlePlus


In [ ]:
z=[]
z=list(zip(news_final['IDLink'],news_final['Title'],news_final['Headline'],news_final['Source'],news_final['Topic'],news_final['LinkedIn']))

z1=sorted(z, key=lambda t: t[5])

labels = ['IDLink','Title', 'Headline','Source', 'Topic', 'SharesOfLinkedIn']
df_linkedIn = pd.DataFrame.from_records(z1, columns=labels)

def label_emotion (row):
   if row['SharesOfLinkedIn'] >=0 and row['SharesOfLinkedIn'] <=100  :
      return 1
   if row['SharesOfLinkedIn'] > 100 and row['SharesOfLinkedIn']<=2000:
      return 2
   if row['SharesOfLinkedIn'] > 2000 and row['SharesOfLinkedIn']<=6000:
      return 3
   if row['SharesOfLinkedIn'] >6000 and row['SharesOfLinkedIn'] <=6400:
      return 4
df_linkedIn['category_LinkedIn'] = df_linkedIn.apply (lambda row: label_emotion (row),axis=1)

df_linkedIn

In [ ]:
from keras.preprocessing.text import one_hot
# integer encode the documents
vocab_size = 100
encoded_docs = [one_hot(d, vocab_size) for d in df_facebook['Title']]

In [ ]:
res = max(encoded_docs, key = len)
len(res)

In [ ]:
from keras.utils import pad_sequences
max_length = 5
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
padded_docs

In [ ]:
x = padded_docs
y = df_facebook['category_facebook']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)


In [ ]:
from sklearn.linear_model import LogisticRegression
#Fit logistic regression to the training set
classifier = LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)

In [ ]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

In [ ]:
y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from keras.preprocessing.text import one_hot
vocab_size = 100
encoded_docs = [one_hot(d, vocab_size) for d in df_googlePlus['Title']]

In [ ]:
res = max(encoded_docs, key = len)
len(res)

In [ ]:
from keras.utils import pad_sequences
max_length = 25
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
from sklearn.linear_model import LogisticRegression
x = padded_docs
y = df_googlePlus['category_GooglePlus']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
#Fit logistic regression to the training set
classifier = LogisticRegression(random_state=1)
classifier.fit(x_train,y_train)

In [ ]:
#Predicting the test set results and creating confusion matrix
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from keras.preprocessing.text import one_hot#import has not been moved from here for better understandability of the code
# integer encode the documents
vocab_size = 100
encoded_docs = [one_hot(d, vocab_size) for d in df_linkedIn['Title']]

In [ ]:
res = max(encoded_docs, key = len)
len(res)

In [ ]:
"""The sequences have different lengths and Keras prefers inputs to be vectorized and all inputs to have the same length. 
We will pad all input sequences to have the length of 4.
Again, we can do this with a built in Keras function, in this case the pad_sequences() function.
pad documents to a max length of 4 words
"""
from keras.utils import pad_sequences
max_length = 25
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
from sklearn.linear_model import LogisticRegression
x = padded_docs
y = df_linkedIn['category_LinkedIn']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
#Fit logistic regression to the training set
classifier = LogisticRegression(random_state=1)
classifier.fit(x_train,y_train)

In [ ]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))